# Graph API Module Example Notebook

This notebook creates 3 tables (users, m365_app_user_detail and teams_acivity_user_details) into a new Spark database called graphapi. 


### Provision storage accounts

The storage account variable has to be changed to the name of the storage account associated with your Azure resource group.

In [1]:
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DoubleType, ArrayType
from pyspark.sql.functions import *
from pyspark.sql.window import Window


# data lake and container information
storage_account = 'stoeahybriddev2'
use_test_env = False

if use_test_env:
    stage1np = 'abfss://test-env@' + storage_account + '.dfs.core.windows.net/stage1np'
    stage2np = 'abfss://test-env@' + storage_account + '.dfs.core.windows.net/stage2np'
else:
    stage1np = 'abfss://stage1np@' + storage_account + '.dfs.core.windows.net'
    stage2np = 'abfss://stage2np@' + storage_account + '.dfs.core.windows.net'

StatementMeta(medium2, 2, 1, Finished, Available)

### Load Raw Data from Lake
To ensure that that the right tables are loaded, confirm that the file paths match your data lake storage containers. 

The top code-block defines the schema of how each of the stage 1 JSON files are stored.

In [2]:
# schemas for each of the JSON files into tables
user_schema = StructType(fields=[
    StructField('value', ArrayType(
        StructType([
            StructField('surname', StringType(), False),
            StructField('givenName', StringType(), False),
            StructField('userPrincipalName', StringType(), False),
            StructField('id', StringType(), False)
        ])
    ))
])

m365_app_user_details_schema = StructType(fields=[
    StructField('value', ArrayType(
        StructType([
            StructField('reportRefreshDate', StringType(), False),
            StructField('userPrincipalName', StringType(), False),
            StructField('lastActivityDate', StringType(), False),
            StructField('reportPeriod', StringType(), False),
            StructField('excel', StringType(), False),
            StructField('excelWeb', StringType(), False),
            StructField('outlook', StringType(), False),
            StructField('outlookWeb', StringType(), False),
            StructField('powerPoint', StringType(), False),
            StructField('powerPointWeb', StringType(), False),
            StructField('teams', StringType(), False),
            StructField('teamsWeb', StringType(), False),
            StructField('word', StringType(), False),
            StructField('wordWeb', StringType(), False),
            StructField('details', StringType())
        ])
    ))
])

teams_activity_user_details_schema = StructType(fields=[
    StructField('value', ArrayType(
        StructType([
            StructField('reportRefreshDate', StringType(), False),
            StructField('reportPeriod', StringType(), False),
            StructField('userPrincipalName', StringType(), False),
            StructField('privateChatMessageCount', IntegerType(), False),
            StructField('teamChatMessageCount', IntegerType(), False),
            StructField('meetingsAttendedCount', IntegerType(), False),
            StructField('meetingCount', IntegerType(), False),
            StructField('audioDuration', StringType(), False),
        ])
    ))
])

StatementMeta(medium2, 2, 2, Finished, Available)

In [5]:
# load needed tables from JSON data lake storage
dfUsersRaw = spark.read.format('json').load(f'{stage1np}/GraphAPI/users.json', schema=user_schema)
dfM365UserActivityRaw = spark.read.format('json').load(f'{stage1np}/GraphAPI/m365_app_user_detail.json', schema=m365_app_user_details_schema)
dfTeamsUserActivityRaw = spark.read.format('json').load(f'{stage1np}/GraphAPI/teams_activity_user_details.json', schema=teams_activity_user_details_schema)

StatementMeta(medium2, 2, 5, Finished, Available)

## 1. Users table
Contains all users (students and teachers) at a school-system level

** Databases and tables used: **

 - None 
 
**JSON files used:**

- users.json

**Database and table created:**

1. Spark DB: graphapi
- Table: users

In [7]:
dfUsersRaw = dfUsersRaw.select(explode('value').alias('exploded_values')).select("exploded_values.*")

StatementMeta(medium2, 2, 7, Finished, Available)

## Write Data Back to Lake

In [ ]:
# write back to the lake in stage 2 ds2_main directory
dfUsersRaw.write.format('parquet').mode('overwrite').save(stage2np + '/GraphAPI/users')

### Load to Spark DB

In [ ]:
# Create spark db to allow for access to the data in the delta-lake via SQL on-demand.
# This is only creating metadata for SQL on-demand, pointing to the data in the delta-lake.
# This also makes it possible to connect in Power BI via the azure sql data source connector.
def create_spark_db(db_name, source_path):
    spark.sql(f'CREATE DATABASE IF NOT EXISTS {db_name}')
    spark.sql(f"DROP TABLE IF EXISTS {db_name}.users")
    spark.sql(f"create table if not exists {db_name}.users using PARQUET location '{source_path}/users'")
    
create_spark_db('graphapi', stage2np + '/GraphAPI/users')

## 2. M365_app_user_detail table
Contains a sample m365 table to support data analysis in a Power BI dashboard.

**Databases and tables used:**
- None

**JSON files used:**
- m365_app_user_detail.json

**Databases and tables created:**

1. Spark DB: graphapi
- Table: m365_app_user_detail

In [ ]:
dfM365UserActivityRaw = dfM365UserActivityRaw.select(explode('value').alias('exploded_values')).select("exploded_values.*")

In [ ]:
# This is a roundabout method (taking in details as a string rather than array), but nontheless - it works for now.

# Isolate and allocate data from "details" to their respective columns
splitDF = dfM365UserActivityRaw.withColumn('reportPeriod', split(col('details'), ',').getItem(0)).withColumn('outlook', split(col('details'), ',').getItem(5)).withColumn('word', split(col('details'), ',').getItem(6)).withColumn('excel', split(col('details'), ',').getItem(7)).withColumn('powerPoint', split(col('details'), ',').getItem(8)).withColumn('teams', split(col('details'), ',').getItem(10)).withColumn('outlookWeb', split(col('details'), ',').getItem(29)).withColumn('wordWeb', split(col('details'), ',').getItem(30)).withColumn('excelWeb', split(col('details'), ',').getItem(31)).withColumn('powerPointWeb', split(col('details'), ',').getItem(32)).withColumn('teamsWeb', split(col('details'), ',').getItem(34))
splitDF = splitDF.drop('details')

# Clean the data within each column, to remove excess string pieces
splitDF = splitDF.withColumn('reportPeriod', regexp_replace('reportPeriod', '"reportPeriod":', '')).withColumn('excel', regexp_replace('excel','"excel":', '')).withColumn('excelWeb', regexp_replace('excelWeb','"excelWeb":', '')).withColumn('outlook', regexp_replace('outlook','"outlook":', '')).withColumn('outlookWeb', regexp_replace('outlookWeb','"outlookWeb":', '')).withColumn('powerPoint', regexp_replace('powerPoint','"powerPoint":', '')).withColumn('powerPointWeb', regexp_replace('powerPointWeb','"powerPointWeb":', '')).withColumn('teams', regexp_replace('teams','"teams":', '')).withColumn('teamsWeb', regexp_replace('teamsWeb','"teamsWeb":', '')).withColumn('word', regexp_replace('word','"word":', '')).withColumn('wordWeb', regexp_replace('wordWeb','"wordWeb":', ''))
splitDF = splitDF.withColumn('reportPeriod', regexp_replace('reportPeriod', '\W+', '')).withColumn('teamsWeb', regexp_replace('teamsWeb', '\W+', ''))

# Transform the datatypes of the columns to be accurate to what they represent
splitDF = splitDF.withColumn('reportPeriod', col('reportPeriod').cast("int")).withColumn('excel', col('excel').cast("boolean")).withColumn('excelWeb', col('excelWeb').cast("boolean")).withColumn('outlook', col('outlook').cast("boolean")).withColumn('outlookWeb', col('outlookWeb').cast("boolean")).withColumn('powerPoint', col('powerPoint').cast("boolean")).withColumn('powerPointWeb', col('powerPointWeb').cast("boolean")).withColumn('teams', col('teams').cast("boolean")).withColumn('teamsWeb', col('teamsWeb').cast("boolean")).withColumn('word', col('word').cast("boolean")).withColumn('wordWeb', col('wordWeb').cast("boolean"))

display(splitDF.limit(10))
#splitDF.printSchema
#display(splitDF)

## Write Data Back to Lake

In [ ]:
# write back to the lake in stage 2 ds2_main directory
splitDF.write.format('parquet').mode('overwrite').save(stage2np + '/GraphAPI/m365_app_user_detail')

### Load to Spark DB

In [ ]:
# Create spark db to allow for access to the data in the delta-lake via SQL on-demand.
# This is only creating metadata for SQL on-demand, pointing to the data in the delta-lake.
# This also makes it possible to connect in Power BI via the azure sql data source connector.
def create_spark_db(db_name, source_path):
    spark.sql(f'CREATE DATABASE IF NOT EXISTS {db_name}')
    spark.sql(f"DROP TABLE IF EXISTS {db_name}.m365_app_user_detail")
    spark.sql(f"create table if not exists {db_name}.users using PARQUET location '{source_path}/m365_app_user_detail'")
    
create_spark_db('graphapi', stage2np + '/GraphAPI/m365_app_user_detail')

## 3. Teams_activity_user_details table
Contains a sample Teams table to support data analysis in a Power BI dashboard.

**Databases and tables used:**
- None

**JSON files used:**
- teams_activity_user_details.json

**Databases and tables created:**

1. Spark DB: graphapi
- Table: teams_activity_user_details

In [ ]:
dfTeamsUserActivityRaw = dfTeamsUserActivityRaw.select(explode('value').alias('exploded_values')).select("exploded_values.*")

## Write Data Back to Lake

In [ ]:
# write back to the lake in stage 2 ds2_main directory
dfTeamsUserActivityRaw.write.format('parquet').mode('overwrite').save(stage2np + '/GraphAPI/teams_activity_user_details')

### Load to Spark DB

In [ ]:
# Create spark db to allow for access to the data in the delta-lake via SQL on-demand.
# This is only creating metadata for SQL on-demand, pointing to the data in the delta-lake.
# This also makes it possible to connect in Power BI via the azure sql data source connector.
def create_spark_db(db_name, source_path):
    spark.sql(f'CREATE DATABASE IF NOT EXISTS {db_name}')
    spark.sql(f"DROP TABLE IF EXISTS {db_name}.teams_activity_user_details")
    spark.sql(f"create table if not exists {db_name}.users using PARQUET location '{source_path}/teams_activity_user_details'")
    
create_spark_db('graphapi', stage2np + '/GraphAPI/teams_activity_user_details')